<a href="https://colab.research.google.com/github/segio-165/MY-ACHIEVEMENT-2026/blob/main/AFRISENTI%20CHEETAH_BASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/EverlynAsiko/lm-evaluation-harness.git

Cloning into 'lm-evaluation-harness'...
remote: Enumerating objects: 43923, done.
remote: Total 43923 (delta 0), reused 0 (delta 0), pack-reused 43923 (from 1)
Receiving objects: 100% (43923/43923), 27.94 MiB | 17.47 MiB/s, done.
Resolving deltas: 100% (28742/28742), done.


In [2]:
!cd lm-evaluation-harness; pip install -e .

Obtaining file:///content/lm-evaluation-harness
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 12.9 MB/s eta 0:00:00
  Building editable for lm_eval (pyproject.toml) ... done
  Created wheel for lm_eval: filename=lm_eval-0.4.9.1-0.editable-py3-none-any.whl size=27325 sha256=dac3eed0b8f2bd3ccfe0459965f0de14d902ec1bd066c2cbe48e22bdc9a3bb9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-957nmncv/wheels/8b/3b/a3/4f94

In [3]:
!pip install transformers accelerate sentencepiece
# Cheetah-base needs specific weights handling
!pip install einop

In [19]:
import os
import sys

# 1. Uninstall the conflicting versions
!pip uninstall -y transformers lm-evaluation-harness

# 2. Force a stable transformers version (v4.44.2 is highly stable for Cheetah/mT5)
!pip install transformers==4.44.2 accelerate sentencepiece einops

# --- Start of modifications to lm-evaluation-harness source files ---

# 3. Modify lm-evaluation-harness/__init__.py to prevent hf_audiolm import
init_file_path = "/content/lm-evaluation-harness/lm_eval/models/__init__.py"
if os.path.exists(init_file_path):
    with open(init_file_path, "r") as f:
        content_init = f.read()
    # Ensure the modification is applied only once or replace existing comment
    if "#hf_audiolm, # Commented out by Colab Agent to fix transformers import issue" not in content_init:
        new_content_init = content_init.replace("    hf_audiolm,", "    #hf_audiolm, # Commented out by Colab Agent to fix transformers import issue")
        with open(init_file_path, "w") as f:
            f.write(new_content_init)
        print(f"Modified {init_file_path}: Commented out hf_audiolm import.")
    else:
        print(f"{init_file_path} already modified: hf_audiolm import commented out.")
else:
    print(f"Warning: {init_file_path} not found. Skipping modification.")

# 4. Modify lm-evaluation-harness/hf_audiolm.py to add try-except block
hf_audiolm_file_path = "/content/lm-evaluation-harness/lm_eval/models/hf_audiolm.py"
if os.path.exists(hf_audiolm_file_path):
    with open(hf_audiolm_file_path, "r") as f:
        content_audiolm = f.read()

    problematic_line = "AUTO_MODEL_CLASS = transformers.Qwen2AudioForConditionalGeneration"
    replacement_code = f"try:\n    {problematic_line}\nexcept AttributeError:\n    print('Warning: Qwen2AudioForConditionalGeneration not found, HFAUDIOLMQWEN model might be unavailable.')\n    AUTO_MODEL_CLASS = None # Set to None or a dummy class if not found\n"

    if problematic_line in content_audiolm and "try:" not in content_audiolm: # Check if line exists and not already wrapped
        new_content_audiolm = content_audiolm.replace(problematic_line, replacement_code)
        with open(hf_audiolm_file_path, "w") as f:
            f.write(new_content_audiolm)
        print(f"Modified {hf_audiolm_file_path}: Added try-except around Qwen2AudioForConditionalGeneration import.")
    else:
        print(f"Warning: {problematic_line} not found or already wrapped in {hf_audiolm_file_path}. Skipping modification.")
else:
    print(f"Error: {hf_audiolm_file_path} not found. Cannot apply fix for hf_audiolm.py.")

# --- End of modifications to lm-evaluation-harness source files ---

# 5. Re-install lm-evaluation-harness in editable mode (now that files are modified)
%cd /content/lm-evaluation-harness
!pip install -e .
%cd /content

# Clear Python's module cache for relevant packages to ensure the newly installed/modified versions are loaded
if 'transformers' in sys.modules:
    del sys.modules['transformers']
if 'lm_eval' in sys.modules:
    del sys.modules['lm_eval']
    # Attempt to clear submodules as well, if any were loaded
    for module_name in list(sys.modules.keys()):
        if module_name.startswith('lm_eval.'):
            del sys.modules[module_name]


Found existing installation: transformers 4.44.2
Uninstalling transformers-4.44.2:
  Successfully uninstalled transformers-4.44.2
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
/content/lm-evaluation-harness/lm_eval/models/__init__.py already modified: hf_audiolm import commented out.
/content/lm-evaluation-harness
Obtaining file:///content/lm-evaluation-harness
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lm_eval (pyproject.toml) ... done
  Created wheel for lm_eval: filename=lm_eval-0.4.9.1-0.editable-py3-none-any.whl size=27325 sha256=c20d20313ba22952de27ac63dfb5bf2c80da2e01d2f3c09d17e5f104ad2beef0
  Stored in directory: /tmp/pip-ephem-wheel-cache-5l570nuh/wheels/8b/3b/a3/4f9491fc7a02bc58df7511185ce9669bec3daf1f8c214b09

In [20]:
# 0-Shot Evaluation
!python -m lm_eval --model hf \
    --model_args "pretrained=UBC-NLP/cheetah-base,dtype=bfloat16,trust_remote_code=True" \
    --tasks afrisenti_swa_prompt_1 \
    --num_fewshot 0 \
    --batch_size 1 \
    --device cuda:0 \
    --output_path ./cheetah_swa_p1_0shot.json

2026-02-11 19:33:08.024174: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770838388.057212   10267 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770838388.067929   10267 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770838388.095020   10267 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770838388.095051   10267 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770838388.095059   10267 computation_placer.cc:177] computation placer alr

In [6]:
# 1. Upgrade transformers to the latest version directly from GitHub
# This ensures you have the Qwen2Audio and Vision2Seq classes
!pip install -U git+https://github.com/huggingface/transformers.git

# 2. Re-install the harness with the HF extra
!pip install -U "lm_eval[hf]"

# 3. Install other required dependencies for Cheetah
!pip install accelerate sentencepiece einops

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-5oew90v3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-5oew90v3
  Resolved https://github.com/huggingface/transformers.git to commit ae05b2ae619aa28fdfdcb8244009d585b7e1fed7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 107.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-5.2.0.dev0-py3-none-any.whl size=11420307 sha256=d980e7df59f0f633b243970de05ad8b79b5c907acac39767f0b2f7e22bd32983
  Stored in directory: /tmp/pip-ephem-wheel-cache-3ntdmbem/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Foun

In [21]:
# 0-Shot Evaluation
!python -m lm_eval --model hf \
    --model_args "pretrained=UBC-NLP/cheetah-base,dtype=bfloat16,trust_remote_code=True" \
    --tasks afrisenti_swa_prompt_1 \
    --num_fewshot 0 \
    --batch_size 1 \
    --device cuda:0 \
    --output_path ./cheetah_swa_p1_0shot.json

2026-02-11 19:35:24.597478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770838524.618337   10947 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770838524.625355   10947 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770838524.645792   10947 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770838524.645817   10947 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770838524.645823   10947 computation_placer.cc:177] computation placer alr

To access gated models from Hugging Face, you need to log in with your Hugging Face token. You can generate a token from your [Hugging Face profile settings](https://huggingface.co/settings/tokens).

In [11]:
# Ensure huggingface_hub is installed
!pip install huggingface_hub

# Log in to Hugging Face programmatically. You will be prompted to enter your token.
from huggingface_hub import login
login()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [22]:
# 0-Shot Evaluation
!python -m lm_eval --model hf \
    --model_args "pretrained=UBC-NLP/cheetah-base,dtype=bfloat16,trust_remote_code=True" \
    --tasks afrisenti_swa_prompt_1 \
    --num_fewshot 0 \
    --batch_size 1 \
    --device cuda:0 \
    --output_path ./cheetah_swa_p1_0shot.json

2026-02-11 19:37:22.278745: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770838642.300085   11526 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770838642.307009   11526 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770838642.324684   11526 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770838642.324708   11526 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770838642.324712   11526 computation_placer.cc:177] computation placer alr

In [18]:
import os

hf_audiolm_file_path = "/content/lm-evaluation-harness/lm_eval/models/hf_audiolm.py"

if os.path.exists(hf_audiolm_file_path):
    with open(hf_audiolm_file_path, "r") as f:
        content = f.read()

    # Modify the content to add a try-except block around the problematic import
    # This is a robust way to prevent the AttributeError if the class is not found
    problematic_line = "AUTO_MODEL_CLASS = transformers.Qwen2AudioForConditionalGeneration"
    replacement_code = f"try:\n    {problematic_line}\nexcept AttributeError:\n    print('Warning: Qwen2AudioForConditionalGeneration not found, HFAUDIOLMQWEN model might be unavailable.')\n    AUTO_MODEL_CLASS = None # Set to None or a dummy class if not found\n"

    new_content = content.replace(problematic_line, replacement_code)

    if new_content != content:
        with open(hf_audiolm_file_path, "w") as f:
            f.write(new_content)
        print(f"Modified {hf_audiolm_file_path}: Added try-except around Qwen2AudioForConditionalGeneration import.")
    else:
        print(f"Warning: {problematic_line} not found in {hf_audiolm_file_path}. Skipping modification.")
else:
    print(f"Error: {hf_audiolm_file_path} not found. Cannot apply fix.")


Modified /content/lm-evaluation-harness/lm_eval/models/hf_audiolm.py: Added try-except around Qwen2AudioForConditionalGeneration import.


In [17]:
import os

init_file_path = "/content/lm-evaluation-harness/lm_eval/models/__init__.py"
if os.path.exists(init_file_path):
    with open(init_file_path, "r") as f:
        content = f.read()
    print(f"Content of {init_file_path}:\n{content}")
else:
    print(f"Error: {init_file_path} not found.")


Content of /content/lm-evaluation-harness/lm_eval/models/__init__.py:
from . import (
    anthropic_llms,
    api_models,
    dummy,
    gguf,
    #hf_audiolm, # Commented out by Colab Agent to fix transformers import issue
    hf_steered,
    hf_vlms,
    huggingface,
    ibm_watsonx_ai,
    mamba_lm,
    nemo_lm,
    neuron_optimum,
    openai_completions,
    optimum_ipex,
    optimum_lm,
    sglang_causallms,
    sglang_generate_API,
    textsynth,
    vllm_causallms,
    vllm_vlms,
)


# TODO: implement __all__


try:
    # enable hf hub transfer if available
    import hf_transfer  # type: ignore # noqa
    import huggingface_hub.constants  # type: ignore

    huggingface_hub.constants.HF_HUB_ENABLE_HF_TRANSFER = True
except ImportError:
    pass



After successfully logging in, please re-run the evaluation cell (`PXaxeHQhBrFK` or `YSLh2L6GFa5x`).